In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from constants import (
    NUM_DAYS,
    NUM_EMPLOYEES,
    OUTPUT_FILE,
)
from schedule_generator import ScheduleGenerator
from schedule_validator import ScheduleValidator
from schedule_model import ScheduleModel

In [3]:
# Генерация расписания
schedule_generator = ScheduleGenerator(num_employees=NUM_EMPLOYEES, num_days=NUM_DAYS)
generated_schedule = schedule_generator.generate_schedule(shuffle_rows=True)
schedule_df = pd.DataFrame(
    generated_schedule,
    columns=[f'Day_{i+1}' for i in range(NUM_DAYS)]
)

# Сохранение расписания
schedule_df.to_csv(OUTPUT_FILE, index=False)

In [4]:
# Подготовка данных для обучения
schedule = schedule_df.values
noisy_schedule = ScheduleGenerator.add_noise_to_schedule(schedule, noise_level=0.05)

# Преобразование данных
x_train = noisy_schedule.reshape(1, -1)
y_train = schedule.flatten()
print("x_train sample:", x_train[0][:20])  # Первые 10 значений x_train
print("y_train sample:", y_train[:20])

# Преобразование в тензоры
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

x_train sample: [2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1]
y_train sample: [2 0 0 1 1 1 1 1 0 0 2 2 2 2 2 0 0 1 1 1]


In [5]:
# Обучение модели
model = ScheduleModel()
epochs = 100

criterion = nn.CrossEntropyLoss() # Cross-entropy loss для многоклассовой классификации
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(x_train.view(x_train.size(0), -1)) # Преобразование в 2D тензор

    loss = criterion(outputs.permute(0, 2, 1, 3).reshape(-1, 3), y_train.view(-1)) # Переформатирование тензоров
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

torch.save(model.state_dict(), 'shift_scheduler_model.pth')

Epoch 1/100, Loss: 1.1089338064193726
Epoch 2/100, Loss: 4.840848445892334
Epoch 3/100, Loss: 5.1858601570129395
Epoch 4/100, Loss: 3.764955520629883
Epoch 5/100, Loss: 3.391624927520752
Epoch 6/100, Loss: 2.390517473220825
Epoch 7/100, Loss: 1.8579884767532349
Epoch 8/100, Loss: 1.623233675956726
Epoch 9/100, Loss: 1.2572869062423706
Epoch 10/100, Loss: 0.956608772277832
Epoch 11/100, Loss: 0.6798754334449768
Epoch 12/100, Loss: 0.6558136343955994
Epoch 13/100, Loss: 0.4362602233886719
Epoch 14/100, Loss: 0.3869820833206177
Epoch 15/100, Loss: 0.3589375615119934
Epoch 16/100, Loss: 0.27553871273994446
Epoch 17/100, Loss: 0.2718978524208069
Epoch 18/100, Loss: 0.1772843599319458
Epoch 19/100, Loss: 0.0988670140504837
Epoch 20/100, Loss: 0.08012670278549194
Epoch 21/100, Loss: 0.06218944862484932
Epoch 22/100, Loss: 0.03825996443629265
Epoch 23/100, Loss: 0.02637270838022232
Epoch 24/100, Loss: 0.024127982556819916
Epoch 25/100, Loss: 0.05260690301656723
Epoch 26/100, Loss: 0.0197266377

In [6]:
# Получение предсказания
model.eval()
with torch.no_grad():
    test_input = torch.zeros((1, NUM_EMPLOYEES * NUM_DAYS))  # Тестовый ввод
    schedule = model(test_input).numpy()
    schedule = np.argmax(schedule, axis=-1)[0]  # Получаем метки классов

print("Предсказанное расписание:")
print(schedule[0][:20])

Предсказанное расписание:
[2 2 0 2 0 2 2 2 2 0 1 2 1 2 0 1 0 2 2 1]


In [7]:
# Проверка расписания
validator = ScheduleValidator(schedule)
validation_results = validator.validate_all_rules()

for rule, passed in validation_results.items():
    status = True if passed else False
    print(f"{rule}: {status}")

Минимум два выходных на неделе: False
Выходные идут подряд: False
Минимум 2 сотрудника на смене: True
Нет утренней после вечерней: False
Смена утро/вечер каждую неделю: False
Не более 5 рабочих дней подряд: False


In [ ]:
# Дозапись в файл с расписанием
if all(validation_results.values()): # Если все проверки пройдены успешно
    schedule_df.to_csv(OUTPUT_FILE, mode='a', header=False, index=False)